# Developing and Executing Error-Mitigated NISQ Algorithms Across Devices and Simulators

Cristina Cirstoiu &rarr; <cristina.cirstoiu@quantinuum.com>, Dan Mills &rarr; <daniel.mills@quantinuum.com> - Quantinuum

## Part Three

We will cover:
1. Introduction and getting started with Qermit.
2. Basic error-mitigation tasks; `MitRes` and `MitEx`.
3. Out of the box error-mitigation with Qermit.
4. Advanced use of Qermit to fine tune and combine error-mitigaiton schemes.
5. Developing new error-mitigation schemes.

# Error-Mitigation & Qermit

Noise:
- Noisy Intermediate-Scale Quantum &rarr; low numbers of qubits and high error rates.
- Error correction, break encryption, Grover's search &rarr; ❌

Error-Mitigation:
- Trade reduced noise for increased circuit shots.
- Moderate to no increase in qubit requirements, unlike with error-correction.

## Qermit

Open-source python package for the design and execution of digital error-mitigation: 
- Supports a wide range of error-mitigation methods.
- Has a modular graph-based software design.

By being implemented using TKET, Qermit is platform-agnostic, so may be used:
- with a wide range of quantum hardware.
- in conjunction with several common quantum software development kits.

Protocols presently available through Qermit include several variations upon: 
- ZNE, CDR, and PEC which mitigate for errors in expectation value calculations.
- Error-mitigation based on frame randomisation, and correction through characterisation of State Preparation And Measurement (SPAM) errors. 

Qermit provides a common interface to this selection of error mitigation schemes.

Graph based architecture takes advantage of the modularity of error-mitigation schemes:
- Sub-processes: circuit execution, circuit modification, model fitting, etc. 
- Vertices may be amended to adapt the protocol. 
- Sub-graphs and graphs may be reused and combined. 

## Getting Started
<p><center> <code> pip install qermit </code> </center></p>
<p>Documentation and examples &rarr; <a href="www.qerm.it">www.qerm.it</a>.</p>
<p>Repository and manual &rarr; <a href="https://github.com/CQCL/qermit">https://github.com/CQCL/Qermit</a>.</p>

# Shot Count Experiment

There are two types of error mitigation methods in Qermit: 
- `MitRes` &rarr; modify the distribution of shots retrieved from a backend.
- `MitEx` &rarr return a modified expectation value estimator of some observable.

MitRes and MitEx object may perform any modification of this form, or none.

- Now &rarr; experiments where the output is a collection of shots.
- Later &rarr; experiments where the output is an expectation value.

We will see experiments:
- Conducted in raw TKET, and the equivalent in Qermit.
- With and without errors.
- Where errors have been mitigated by a Qermit `MitRes`.

## Ideal Shot Count

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

circ = Circuit(2,2).H(0).CX(0,1).measure_all()
render_circuit_jupyter(circ)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def plot_state_probs(state):
    state_dict = {'State':[i for i in range(len(result_state))], 'Probability':abs(state)**2}
    state_df = pd.DataFrame(state_dict)
    sns.catplot(x='State', y='Probability', kind='bar', data=state_df, aspect = 3, height=2)
    plt.show()
    
def plot_counts(counts):
    counts_record = [{"State":state, "Count":count} for state, count in counts.items()]
    count_df = pd.DataFrame().from_records(counts_record)
    sns.catplot(x='State', y='Count', kind='bar', data=count_df, aspect = 3, height=2)
    plt.show()

In [ ]:
from pytket.extensions.qiskit import AerBackend

n_shots = 10000
ideal_backend = AerBackend()
result = ideal_backend.run_circuit(circ, n_shots=n_shots)
plot_counts(result.get_counts())

In [ ]:
from qermit import MitRes, CircuitShots

circ_shots_list = [CircuitShots(circ, n_shots)]
ideal_mitres = MitRes(ideal_backend)

result_list = ideal_mitres.run(circ_shots_list)
result_counts = result_list[0].get_counts()
plot_counts(result_counts)

## MitRes TaskGraph

In [ ]:
ideal_mitres.get_task_graph()

`MitRes` and `MitEx` objects are constructed as dataflow graphs, called a `TaskGraph`. 
- Each node of a `TaskGraph` is a `MitTask` object; itself a function that computes some step or sub-process of an error mitigation protocol. 
- Edges of the graph move data between `MitTask` objects. 
- When `run` is called, `MitTask`s are ordered and run sequentially.

## SPAM + Depolarising Noise Shot Count

In [ ]:
import qiskit.providers.aer.noise as noise

def depolarizing_noise_model(n_qubits, prob_1, prob_2, prob_ro):

    noise_model = noise.NoiseModel()

    error_2 = noise.depolarizing_error(prob_2, 2)
    for edge in [[i,j] for i in range(n_qubits) for j in range(i)]:
        noise_model.add_quantum_error(error_2, ['cx'], [edge[0], edge[1]])
        noise_model.add_quantum_error(error_2, ['cx'], [edge[1], edge[0]])

    error_1 = noise.depolarizing_error(prob_1, 1)
    for node in range(n_qubits):
        noise_model.add_quantum_error(error_1, ['h', 'rx', 'rz', 'u'], [node])
        
    probabilities = [[1-prob_ro, prob_ro],[prob_ro, 1-prob_ro]]
    error_ro = noise.ReadoutError(probabilities)
    for i in range(n_qubits):
        noise_model.add_readout_error(error_ro, [i])
        
    return noise_model

In [ ]:
from qermit.taskgraph.mitex import MitEx, gen_compiled_MitRes
from pytket.extensions.qiskit import AerBackend

noisy_backend = AerBackend(depolarizing_noise_model(5, 0.001, 0.01, 0.05))
noisy_mitres = gen_compiled_MitRes(noisy_backend, optimisation_level=0)

noisy_result_list = noisy_mitres.run(circ_shots_list)
noisy_result_counts = noisy_result_list[0].get_counts()
plot_counts(noisy_result_counts)

## SPAM Error-Mitigation with Qermit

In [ ]:
from qermit.spam import gen_UnCorrelated_SPAM_MitRes

spam_mr = gen_UnCorrelated_SPAM_MitRes(noisy_backend, n_shots)
spam_result_list = spam_mr.run(circ_shots_list)
spam_result_counts = spam_result_list[0].get_counts()

In [ ]:
plot_counts(noisy_result_counts)
plot_counts(spam_result_counts)

## SPAM MitRes Task Graph

In [ ]:
spam_mr.get_task_graph()

# Expectation Value Experiment

Experiments where the output is an expectation value.

We will see experiments:
- Conducted in raw TKET, and the equivalent in Qermit.
- With and without errors.
- Where errors have been mitigated by a Qermit `MitEx`.

## Ideal Expectation Value

In [ ]:
import numpy as np
from scipy.stats import unitary_group
from pytket.circuit import Unitary2qBox

def random_circ(n_qubits: int, depth: int, seed:int = None) -> Circuit:
    
    np.random.seed(seed)

    c = Circuit(n_qubits)

    for _ in range(depth):

        qubits = np.random.permutation([i for i in range(n_qubits)])
        qubit_pairs = [[qubits[i], qubits[i + 1]] for i in range(0, n_qubits - 1, 2)]

        for pair in qubit_pairs:

            # Generate random 4x4 unitary matrix.
            SU4 = unitary_group.rvs(4)  # random unitary in SU4
            SU4 = SU4 / (np.linalg.det(SU4) ** 0.25)
            SU4 = np.matrix(SU4)

            # Add gate corresponding to unitary.
            c.add_unitary2qbox(Unitary2qBox(SU4), *pair)

    return c

In [ ]:
from pytket.pauli import Pauli, QubitPauliString
from pytket import Qubit 
from pytket.extensions.qiskit import AerStateBackend

n_qubits = 4
rand_circ = random_circ(n_qubits,n_qubits,seed=23126)
render_circuit_jupyter(rand_circ)

In [ ]:
ideal_circuit = rand_circ.copy()
ideal_circuit = ideal_backend.get_compiled_circuit(ideal_circuit)

qps = QubitPauliString([Qubit(i) for i in range(n_qubits)], [Pauli.Z for i in range(n_qubits)])

print(f"Ideal expectation: {ideal_backend.get_pauli_expectation_value(ideal_circuit, qps)}")

In [ ]:
from qermit import MitEx
from pytket.utils import QubitPauliOperator
from qermit import AnsatzCircuit, SymbolsDict, ObservableExperiment, ObservableTracker

ideal_mitex = MitEx(ideal_backend)

qpo = QubitPauliOperator({qps:1})
obs_exp = ObservableExperiment(AnsatzCircuit(rand_circ, n_shots, SymbolsDict()), ObservableTracker(qpo))
obs_exp_list = [obs_exp]

ideal_expectation = ideal_mitex.run(obs_exp_list)
print(f"Ideal expectation: {ideal_expectation[0]}")

## MitEx Task Graph

In [ ]:
ideal_mitex.get_task_graph()

## Noisy Expectation Value

In [ ]:
noisy_mitex = MitEx(noisy_backend)

noisy_expectation = noisy_mitex.run(obs_exp_list)
print(f"Noisy expectation: {noisy_expectation[0]}")

## ZNE Error-Mitigation with Qermit

In [ ]:
from qermit.zero_noise_extrapolation import gen_ZNE_MitEx, Fit, Folding

zne_me = gen_ZNE_MitEx(backend=noisy_backend, 
                       noise_scaling_list=[9,7,5,3], 
                       fit_type=Fit.exponential, 
                       folding_type=Folding.circuit,
                       show_fit=True)

In [ ]:
import seaborn as sns 
sns.set_style("whitegrid")

In [ ]:
zne_me.run(obs_exp_list)

## ZNE MitEx Task Graph

In [ ]:
zne_me.get_task_graph()

## ZNE Options and Design

There are several alternatives when building a ZNE task graph. 

In [ ]:
alt_zne_me = gen_ZNE_MitEx(backend=noisy_backend, 
                       noise_scaling_list=[5,4,3,2], 
                       fit_type=Fit.richardson, 
                       folding_type=Folding.gate,
                       show_fit=True)

Your favourites are available:
- Folding: `circuit`, `gate`, `odd_gate`
- Fit: `poly_exponential`, `exponential`, `polynomial`, `linear`, `richardson`

For many `MitEx` objects there are several options to personalise, without interacting directly with the `TaskGraph`.

# Advanced Qermit Use

We have seen:
- `MitRes` and `MitEx` objects.
- Use of predefined `MitRes` and `MitEx` objects, such as SPAM and ZNE.
- Personalised use of predefined objects.

We'll now look at some more advanced use cases:
- Combining error-mitigation protocols.
- Constructing original protocols.

## Combining Error-Mitigation Schemes

In [ ]:
zne_spam_me = gen_ZNE_MitEx(backend=noisy_backend, 
                       noise_scaling_list=[9,7,5,3], 
                       fit_type=Fit.exponential, 
                       show_fit=True,
                       experiment_mitres=spam_mr)

## Combined ZNE SPAM Task Graph

In [ ]:
zne_spam_me.get_task_graph()

In [ ]:
zne_spam_me.run(obs_exp_list)

## Clifford Data Regression

In [ ]:
from qermit.clifford_noise_characterisation import gen_CDR_MitEx

cdr_mitex = gen_CDR_MitEx(device_backend = noisy_backend,
                      simulator_backend = ideal_backend,
                      n_non_cliffords = 5,
                      n_pairs = 3,
                      total_state_circuits = 50)

## CDR Task Graph

In [ ]:
cdr_mitex.get_task_graph()

In [ ]:
cdr_mitex.run(obs_exp_list)

## Developing Original Error-Mitigation Protocols

In [ ]:
noisy_backend = AerBackend(depolarizing_noise_model(5, 0, 0, 0.25))
noisy_mitres = gen_compiled_MitRes(noisy_backend, optimisation_level=0)

In [ ]:
circ = Circuit(2)
circ.X(1).measure_all()
render_circuit_jupyter(circ)

In [ ]:
circ_shots_list = [CircuitShots(circ, n_shots)]

result_list = noisy_mitres.run(circ_shots_list)
result_counts = result_list[0].get_counts()
plot_counts(result_counts)

## Find Zero MitRes TaskGraph

In [ ]:
from pytket import OpType

def gen_find_zeros_task():
    
    def task(obj, circ_shots_list):
        
        zero_loc_list = []
        circ_size_list = []
        
        for circ_shot in circ_shots_list:
            not_zero_loc = []
            for command in circ_shot.Circuit.get_commands():
                if not command.op.type == OpType.Measure:
                    not_zero_loc.extend(command.qubits)
            zero_loc = [loc for loc in circ_shot.Circuit.qubits if loc not in not_zero_loc]
            zero_loc_list.append(zero_loc)
            circ_size_list.append(circ_shot.Circuit.n_qubits)
        
        return (circ_shots_list, zero_loc_list, circ_size_list, )
    
    return MitTask(_label="FindZeros", _n_out_wires=3, _n_in_wires=1, _method=task)

In [ ]:
from pytket.backends.backendresult import BackendResult

def gen_remove_ones_task():
    
    def task(obj, result_list, zero_loc_list, circ_size_list) -> Tuple[List[BackendResult]]:
        
        mitigated_result_list = []
        for result, zero_loc, circ_size in zip(result_list, zero_loc_list, circ_size_list):
            
            result_dict = result.to_dict()
            result_shots_array = result_dict['shots']['array']
            
            mitigated_result_shots_array = []
            for result_shots in result_shots_array:
                            
                result_shot_binary = bin(result_shots[0])[2:].zfill(8)[:circ_size][::-1]
                delete_shot = False
                for zero in zero_loc:
                    if result_shot_binary[zero.index[0]] == "1":
                        delete_shot = True
                if not delete_shot:
                    mitigated_result_shots_array.append(result_shots)
            
            result_dict['shots']['array'] = mitigated_result_shots_array
            mitigated_result = BackendResult.from_dict(result_dict)
            mitigated_result_list.append(mitigated_result)

        return (mitigated_result_list, )
    
    return MitTask(_label="RemoveOnes", _n_out_wires=1, _n_in_wires=3, _method=task)

In [ ]:
from qermit.taskgraph.task_graph import TaskGraph
from typing import Tuple, List
from qermit.taskgraph.mittask import MitTask

taskgraph = TaskGraph().from_TaskGraph(noisy_mitres)

taskgraph.add_wire()
taskgraph.add_wire()

taskgraph.prepend(gen_find_zeros_task())
taskgraph.append(gen_remove_ones_task())

find_zero_mitres = MitRes(noisy_mitres).from_TaskGraph(taskgraph)

In [ ]:
find_zero_mitres.get_task_graph()

In [ ]:
result_list = find_zero_mitres.run(circ_shots_list)
result_counts = result_list[0].get_counts()
plot_counts(result_counts)

<center> <h1> Questions </h1> </center>